In [45]:
import pickle
import json
import random
import tensorflow
import tflearn
import numpy
import nltk

In [191]:
with open("intents.json", 'rb') as file:
    data = json.load(file)

In [192]:
data

{'intents': [{'tag': 'cumprimento',
   'patterns': ['Olá', 'Ola', 'Ei', 'Oi', 'Bom dia', 'Boa noite'],
   'responses': ['Olá, bem-vindo ao nosso Chatbot',
    'Bom ver você, bem-vindo ao nosso chatbot',
    'Olá, como posso ajudar?'],
   'context': ['']},
  {'tag': 'adeus',
   'patterns': ['Tchau',
    'Até logo',
    'Bom bate-papo com você, tchau',
    'Até a próxima'],
   'responses': ['Até logo!',
    'Tenha um bom dia',
    'Tchau! Volte logo.',
    'Feliz por ajudar'],
   'context': ['']},
  {'tag': 'obrigado',
   'patterns': ['obrigado',
    'Isso ajuda',
    'Demais obrigado',
    'Obrigado por me ajudar'],
   'responses': ['Feliz por ajudar! ',
    'A qualquer momento! Posso ajudar em algum outro problema?',
    'O prazer é meu! Posso ajudar com algum outro problema?'],
   'context': ['']},
  {'tag': 'valor do IPVA',
   'patterns': ['Qual o valor do IPVA?'],
   'responses': ['Os valores separados do IPVA, da Taxa de Licenciamento Anual e do seguro obrigatório podem ser obtidos

In [193]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

In [194]:
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [195]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

In [196]:
labels = sorted(labels)

In [197]:
words_size = len(words)
words_size

33

In [198]:
training = []
output = []

In [199]:
out_empty = [0 for _ in range(len(labels))]
for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

In [200]:
training = numpy.array(training)
output = numpy.array(output)

In [201]:
tensorflow.reset_default_graph()
tflearn.init_graph(num_cores=4,gpu_memory_fraction=0.5)

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.embedding(net, input_dim=words_size, output_dim=128)
#net = tflearn.lstm(net, 128, dropout=0.8, return_seq=True)
net = tflearn.lstm(net, 128, dropout=0.8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy')

In [ ]:
model = tflearn.DNN(net)
history = model.fit(training, output, n_epoch=300, batch_size=128, show_metric=True, run_id='intents')
#model.save("model.tflearn")

---------------------------------
Run id: intents
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 17
Validation samples: 0
--
Training Step: 1  | time: 13.366s
| Adam | epoch: 001 | loss: 0.00000 - acc: 0.0000 -- iter: 17/17
--
Training Step: 2  | total loss: 1.61296 | time: 0.099s
| Adam | epoch: 002 | loss: 1.61296 - acc: 0.0529 -- iter: 17/17
--
Training Step: 3  | total loss: 1.75723 | time: 0.123s
| Adam | epoch: 003 | loss: 1.75723 - acc: 0.2984 -- iter: 17/17
--
Training Step: 4  | total loss: 1.77807 | time: 0.100s
| Adam | epoch: 004 | loss: 1.77807 - acc: 0.3393 -- iter: 17/17
--
Training Step: 5  | total loss: 1.77982 | time: 0.108s
| Adam | epoch: 005 | loss: 1.77982 - acc: 0.3487 -- iter: 17/17
--
Training Step: 6  | total loss: 1.77724 | time: 0.100s
| Adam | epoch: 006 | loss: 1.77724 - acc: 0.3514 -- iter: 17/17
--
Training Step: 7  | total loss:

Training Step: 58  | total loss: 1.54587 | time: 0.231s
| Adam | epoch: 058 | loss: 1.54587 - acc: 0.3529 -- iter: 17/17
--
Training Step: 59  | total loss: 1.54541 | time: 0.101s
| Adam | epoch: 059 | loss: 1.54541 - acc: 0.3529 -- iter: 17/17
--
Training Step: 60  | total loss: 1.54167 | time: 0.097s
| Adam | epoch: 060 | loss: 1.54167 - acc: 0.3529 -- iter: 17/17
--
Training Step: 61  | total loss: 1.54101 | time: 0.120s
| Adam | epoch: 061 | loss: 1.54101 - acc: 0.3529 -- iter: 17/17
--
Training Step: 62  | total loss: 1.54255 | time: 0.124s
| Adam | epoch: 062 | loss: 1.54255 - acc: 0.3529 -- iter: 17/17
--
Training Step: 63  | total loss: 1.54285 | time: 0.126s
| Adam | epoch: 063 | loss: 1.54285 - acc: 0.3529 -- iter: 17/17
--
Training Step: 64  | total loss: 1.53962 | time: 0.108s
| Adam | epoch: 064 | loss: 1.53962 - acc: 0.3529 -- iter: 17/17
--
Training Step: 65  | total loss: 1.53896 | time: 0.124s
| Adam | epoch: 065 | loss: 1.53896 - acc: 0.3529 -- iter: 17/17
--
Training

In [188]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [189]:
def chat():
    print("Comece a conversar com o bot (digite quit para parar)!")
    while True:
        inp = input("Você: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if results[results_index] > 0.7:
            for tg in data["intents"]:
                if tg["tag"] == tag:
                    responses = tg["responses"]

            print(random.choice(responses))
        else:
            print("Não entendi, tente novamente. ")

In [190]:
chat()

Comece a conversar com o bot (digite quit para parar)!
Você: ola
Não entendi, tente novamente. 
Você: ola
Não entendi, tente novamente. 
Você: olá
Olá, bem-vindo ao nosso Chatbot
Você: tudo bem?
Não entendi, tente novamente. 
Você: pagar o ipva
O prazer é meu! Posso ajudar com algum outro problema?
Você: como faço para pagar o ipva
Os valores separados do IPVA, da Taxa de Licenciamento Anual e do seguro obrigatório podem ser obtidos nesta página da Internet (www.receita.fazenda.df.gov.br) em: Menu Receita / Serviços Cidadão / Veículos, mediante o fornecimento do nº do RENAVAM ou por meio do telefone “156”, opção “3”, ou em uma das unidades do “Na Hora”, ou em uma das Agências/Posto de Atendimento da Receita, mediante o fornecimento do nº da placa e do RENAVAM.
Você: como pagar o ipva anos anteriores
Os valores pagos e a pagar podem ser obtidos nesta página da Internet (www.receita.fazenda.df.gov.br) em: Menu Receita / Serviços Cidadão / Veículos, ou por meio do telefone “156”, opção “3

KeyboardInterrupt: Interrupted by user